In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from tensorflow.keras.layers import concatenate
from deepforest import CascadeForestClassifier
from tensorflow.keras.optimizers import Adam

In [2]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5 (1).csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images.shape)

1000
(1000, 320, 320, 9)


In [3]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)
print(trainImagesX.shape)
print(trainy.shape)

(800, 2)
(200, 2)
(800,)
(200,)
(800, 320, 320, 9)
(800,)


In [13]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [5]:
tf.compat.v1.disable_eager_execution()

In [17]:
def create_cnn_mobilenet ():
     input = Input(shape = (320,320,9))
     x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
     x = BatchNormalization()(x)
     x = ReLU()(x)

     x = mobilnet_block(x, filters = 64, strides = 1)
     x = mobilnet_block(x, filters = 128, strides = 2)
     x = mobilnet_block(x, filters = 128, strides = 1)
     x = mobilnet_block(x, filters = 256, strides = 2)
     x = mobilnet_block(x, filters = 256, strides = 1)
     x = mobilnet_block(x, filters = 512, strides = 2)
     for _ in range (5):
          x = mobilnet_block(x, filters = 512, strides = 1)
     x = mobilnet_block(x, filters = 1024, strides = 2)
     x = mobilnet_block(x, filters = 1024, strides = 1)
     x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
     output = Flatten()(x)
     model = Model(inputs=input, outputs=output)
     return model

In [18]:
mobilenet = create_cnn_mobilenet()
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 320, 320, 9)]     0         
                                                                 
 conv2d_56 (Conv2D)          (None, 160, 160, 32)      2624      
                                                                 
 batch_normalization_108 (B  (None, 160, 160, 32)      128       
 atchNormalization)                                              
                                                                 
 re_lu_108 (ReLU)            (None, 160, 160, 32)      0         
                                                                 
 depthwise_conv2d_52 (Depth  (None, 160, 160, 32)      320       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_109 (B  (None, 160, 160, 32)      128 

In [ ]:
#hist = model.fit(x=trainImagesX, y=trainy,validation_data=(testImagesX, testy),epochs=10)

In [21]:
mobilenetPredict=np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
print(mobilenetPredict.shape)
print(trainAttrX.shape)
trainData = concatenate([mobilenetPredict, trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)

print(trainData.shape)

(800, 40720)
(800, 2)


NotImplementedError: Cannot convert a symbolic tf.Tensor (concatenate_4/concat:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.

In [ ]:
model = CascadeForestClassifier(n_estimators=50,random_state=1,)
model.fit(trainData,trainy)


[2023-12-24 00:49:56.995] Start to fit the model:
[2023-12-24 00:49:56.995] Fitting cascade layer = 0 


[2023-12-24 00:50:06.156] layer = 0  | Val Acc = 99.250 % | Elapsed = 9.161 s
[2023-12-24 00:50:06.169] Fitting cascade layer = 1 
[2023-12-24 00:50:16.237] layer = 1  | Val Acc = 99.375 % | Elapsed = 10.068 s
[2023-12-24 00:50:16.249] Fitting cascade layer = 2 
[2023-12-24 00:50:26.572] layer = 2  | Val Acc = 99.125 % | Elapsed = 10.323 s
[2023-12-24 00:50:26.572] Early stopping counter: 1 out of 2
[2023-12-24 00:50:26.584] Fitting cascade layer = 3 
[2023-12-24 00:50:37.525] layer = 3  | Val Acc = 99.250 % | Elapsed = 10.941 s
[2023-12-24 00:50:37.525] Early stopping counter: 2 out of 2
[2023-12-24 00:50:37.525] Handling early stopping
[2023-12-24 00:50:37.533] The optimal number of layers: 2


In [ ]:
cows = mobilenet.predict(testImagesX)
cows = np.array(cows)
# print(cows)
print(cows.shape)
print(testAttrX.shape)
# testAttrX.resize(122,2)
# print(gasdata)
# print(gasdata.shape)
test = concatenate([cows,testAttrX])


7/7 [==============================] - 4s 445ms/step
(200, 1)
(200, 2)


In [ ]:
from sklearn.metrics  import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
performance = model.predict(test)
performance.round()
actual = []
for value in performance: 
    print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

acutal = np.array(actual)
testy=np.array(testy)
print(actual)
print(testy)
pigs=np.array(testy)
actual.append(0)
actual.append(0)
actual.append(0)
actual.append(0)
actual.append(0)
actual.append(0)
testy=np.append(testy,1)

accuracy = accuracy_score(testy, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

[2023-12-24 00:52:58.320] Start to evalute the model:
[2023-12-24 00:52:58.320] Evaluating cascade layer = 0 
[2023-12-24 00:52:58.552] Evaluating cascade layer = 1 
0
0
0
0
0
0
1
1
0
0
0
0
0
1
1
0
0
0
0
0
1
1
0
1
0
1
1
0
1
1
1
1
1
1
0
0
1
0
0
0
1
0
0
0
1
1
0
0
1
0
0
1
1
0
1
0
1
1
0
0
1
0
0
1
0
0
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
0
1
1
1
1
1
0
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
0
1
1
0
0
1
0
0
1
1
1
0
0
1
0
1
0
0
0
0
0
0
1
1
0
1
1
0
0
1
0
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
1
0
1
0
1
1
0
1
1
0
0
0
1
0
0
1
0
1
1
0
0
1
1
1
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
1
0
1
1
1
1
1
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0